In [28]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import base64

tqdm.pandas()
from wekeypedia import WikipediaPage as Page

In [29]:
name = "Elliot Page"
dead_name = "RWxsZW4="
page_name = "Elliot_Page"
co_date = "2020-12-01"

In [30]:
def format_revid_index(revid, page):
    return f'<a href="https://fr.wikipedia.org/w/index.php?title={page}&diff=prev&oldid={revid}">{revid}</a>'

# qui sont personnes qui énoncent le deadname dans l'article ?

## article

In [31]:
page = Page(page_name, lang='fr')

In [43]:
revisions = (
    pd
    .DataFrame
    .from_records(page.get_revisions_list())
    .assign(
        diff = lambda df: df.revid.progress_apply(lambda x: page.get_diff(x)).fillna(''),
        deleted = lambda df: df['diff'].apply(lambda diff: page.extract_plusminus(diff)['deleted']),
        added = lambda df: df['diff'].apply(lambda diff: page.extract_plusminus(diff)['added']),
    )
    .set_index('revid')
)

revisions

100%|██████████████████████████████████████████████████████████████████████| 500/500 [02:48<00:00,  2.96it/s]


,parentid,user,userid,timestamp,size,sha1,comment,anon,sha1hidden,diff,deleted,added
revid,,,,,,,,,,,,
213863884,213709169,Bessam&Mucho,1860532,2024-04-01T09:47:25Z,58575,cd271df6ebbfd642ba20dba740093504a356f0dc,,NaN,NaN,"<tr>\n <td colspan=""2"" class=""diff-lineno"">Li...",[-],[ ]
213709169,213709114,Kirham,10413,2024-03-26T23:08:57Z,58575,898b3476b0409869e84a3ea4a587790c56bad55a,Annulation de la [[Special:Diff/213709114|modi...,NaN,NaN,"<tr>\n <td colspan=""2"" class=""diff-lineno"">Li...",[],[et connue sous le nom d{{'}}Ellen Page avant ...
213709114,213330191,Atsume Ryuguji,4926922,2024-03-26T23:05:55Z,58441,0cb8696d817d84ebce57e58ffcccf0878bc943eb,il ne faut pas mettre le dead name des personn...,NaN,NaN,"<tr>\n <td colspan=""2"" class=""diff-lineno"">Li...",[et connue sous le nom d{{'}}Ellen Page avant ...,[]
213330191,213326752,Nouill,550436,2024-03-13T22:01:37Z,58575,898b3476b0409869e84a3ea4a587790c56bad55a,/* Longs métrages */,NaN,NaN,"<tr>\n <td colspan=""2"" class=""diff-lineno"">Li...",[],[* 2023 : ''Close to You'' de [[Dominic Savage...
213326752,213151379,OrlodrimBot,1055871,2024-03-13T19:45:39Z,58521,0c92c8684d2fe222b8986912c2474b60768aed24,Retrait de l'icône de semi-protection,NaN,NaN,"<tr>\n <td colspan=""2"" class=""diff-lineno"">Li...",[{{Semi-protection}}],[]
...,...,...,...,...,...,...,...,...,...,...,...,...
178308671,178307602,Aquilons,3367104,2021-01-02T06:28:57Z,42431,144a9118c8bef021132f7ed7605ecf664d3b9754,Annulation de la [[Special:Diff/178307602|modi...,NaN,NaN,"<tr>\n <td colspan=""2"" class=""diff-lineno"">Li...","[canadien, né, connu]","[canadienne, née, connue]"
178307602,178304224,142.119.22.118,0,2021-01-02T04:53:06Z,42427,f8226400bec2d5764b41e8c23b998f92402c357d,Correcteur de pronoms,,NaN,"<tr>\n <td colspan=""2"" class=""diff-lineno"">Li...","[canadienne, née, connue]","[canadien, né, connu]"
178304224,178304147,Triboulet sur une montagne,3069280,2021-01-02T00:40:51Z,42431,144a9118c8bef021132f7ed7605ecf664d3b9754,Annulation de la [[Special:Diff/178304147|modi...,NaN,NaN,"<tr>\n <td colspan=""2"" class=""diff-lineno"">Li...",[Née],[Né]


In [44]:
revisions.to_csv(f'../data/page-revisions/{page_name.lower()}.csv')

In [45]:
def decode(string):
    return base64.b64decode(string).decode("utf-8")

In [46]:
# (
#     revisions
#     .assign(
#         deadnames = lambda df: df['*'].apply(lambda x: str(x).lower().count(decode(dead_name).lower()))
#     )
#     #.reset_index()
#     [['timestamp', 'deadnames']]
#     .assign(
#         date = lambda df: pd.to_datetime(df.timestamp).dt.date
#     )
#     .groupby('date')
#     .agg({'deadnames': 'mean'})
#     .plot
#     .line(
#         figsize=(15,10)
#     )
# )

### depuis le coming out

In [47]:
(
    revisions
    .assign(
        deadname_count = lambda df: df.added.apply(lambda x: ' '.join(x).lower().count(decode(dead_name).lower()))
    )
    # on ne garde que les contributions après son coming out
    .pipe(lambda df: df.query(f'timestamp >= "{co_date}"') if co_date != None else df)
    .groupby('user')
    .deadname_count
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

user
Voxhominis                                45
Nattes à chat                             23
SRLVR                                     16
Triboulet sur une montagne                16
81.185.166.18                             16
Cymbella                                  14
Kvardek du                                12
2A01:CB00:2FA:3400:E80B:16D9:B2A4:D7C3    11
Sherwood6                                  8
Aqw96                                      7
Name: deadname_count, dtype: int64

### depuis le début du sondage

In [48]:
(
    revisions
    .assign(
        deadname_count = lambda df: df.added.apply(lambda x: ' '.join(x).lower().count(decode(dead_name).lower()))
    )
    # début du sondage
    .query('timestamp >= "2024-01-15"')
    .groupby('user')
    .deadname_count
    .sum()
    .sort_values(ascending=False)
    .head(20)
)

user
Triboulet sur une montagne                3
Sherwood6                                 3
Lomita                                    3
MDCCCC                                    2
Esprit Fugace                             2
Kirham                                    2
Vincent Lefèvre                           1
SleaY                                     1
2001:861:5C41:7500:A643:26C4:48FE:6231    1
Chouette bougonne                         1
Cymbella                                  1
Nocta Senestra                            0
Noname666777999                           0
2001:861:540D:79D0:A1CF:6195:43B8:D057    0
Nouill                                    0
Pic-Sou                                   0
Pronoia                                   0
Skimel                                    0
Taniki                                    0
OrlodrimBot                               0
Name: deadname_count, dtype: int64

In [20]:
(
    revisions
    .assign(
        deadname_count = lambda df: df.added.apply(lambda x: ' '.join(x).lower().count(decode(dead_name).lower()))
    )
    # début du sondage
    .query('timestamp >= "2024-01-15"')
    .query('deadname_count > 0')
    [['user', 'deadname_count', 'timestamp']]
    .style
    .format_index(lambda x: format_revid_index(x, page_name))
)

,user,deadname_count,timestamp
revid,,,
213709169,Kirham,1,2024-03-26T23:08:57Z
213101415,Cymbella,1,2024-03-06T08:50:38Z
213101254,MDCCCC,1,2024-03-06T08:44:34Z
213100978,Sherwood6,1,2024-03-06T08:34:24Z
213100895,MDCCCC,1,2024-03-06T08:31:32Z
212939166,Sherwood6,2,2024-03-01T13:33:48Z
212866430,Vincent Lefèvre,1,2024-02-27T23:05:54Z
212814090,Lomita,3,2024-02-26T07:08:15Z
212803201,Triboulet sur une montagne,3,2024-02-25T20:00:54Z


## Discussions

In [21]:
talk = Page(f'Discussion:{page_name}', lang='fr')

In [22]:
talk_revisions = (
    pd
    .DataFrame
    .from_records(talk.get_revisions_list())
    .assign(
        diff = lambda df: df.revid.progress_apply(lambda x: talk.get_diff(x)),
        deleted = lambda df: df['diff'].apply(lambda diff: talk.extract_plusminus(diff)['deleted']),
        added = lambda df: df['diff'].apply(lambda diff: talk.extract_plusminus(diff)['added']),
    )
    .set_index('revid')
)

#talk_revisions

100%|████████████████████████████████████████████████████████████████████| 1161/1161 [07:02<00:00,  2.75it/s]


In [23]:
talk_revisions.to_csv(f'../data/page-revisions/discussion:{page_name.lower()}.csv')

### depuis le coming out

In [24]:
(
    talk_revisions
    .assign(
        deadname_count = lambda df: df.added.apply(lambda x: ' '.join(x).lower().count(decode(dead_name).lower()))
    )
    # on ne garde que les contributions après son coming out
    #.query('timestamp >= "2013-08-22"')
    .groupby('user')
    .deadname_count
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

user
Nattes à chat                 19
Laurent Jerry                 18
Victoire F.                   12
Triboulet sur une montagne    10
GF38storic                     9
Aqw96                          8
Hexasoft                       7
Michel1961                     7
SRLVR                          6
Apollinaire93                  5
Name: deadname_count, dtype: int64

### depuis le début du sondage

In [25]:
(
    talk_revisions
    .assign(
        deadname_count = lambda df: df.added.apply(lambda x: ' '.join(x).lower().count(decode(dead_name).lower()))
    )
    # depuis le sondage
    .query('timestamp >= "2024-01-15"')
    .groupby('user')
    .deadname_count
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

user
GF38storic                    9
JohnNewton8                   2
Sherwood6                     2
Anne Daemon                   1
176.155.30.213                0
OrlodrimBot                   0
Typhoeus                      0
Triboulet sur une montagne    0
Titoo2026                     0
Thontep                       0
Name: deadname_count, dtype: int64

In [26]:
(
    talk_revisions
    .assign(
        deadname_count = lambda df: df.added.apply(lambda x: ' '.join(x).lower().count(decode(dead_name).lower()))
    )
    # depuis le sondage
    .query('timestamp >= "2024-01-15"')
    .query('deadname_count > 0')
    [['user', 'deadname_count', 'timestamp']]
    .style
    .format_index(lambda x : format_revid_index(x, f'Discussion:{page_name}'))
)

,user,deadname_count,timestamp
revid,,,
212863799,Sherwood6,1,2024-02-27T21:40:09Z
212861096,Anne Daemon,1,2024-02-27T20:12:44Z
212846485,GF38storic,4,2024-02-27T10:19:07Z
212830551,JohnNewton8,2,2024-02-26T18:00:16Z
212823412,GF38storic,2,2024-02-26T13:46:27Z
212820314,Sherwood6,1,2024-02-26T11:36:35Z
212793265,GF38storic,1,2024-02-25T14:51:21Z
212790878,GF38storic,2,2024-02-25T13:51:44Z


In [27]:
len(
    talk_revisions
    .assign(
        deadname_count = lambda df: df.added.apply(lambda x: ' '.join(x).lower().count(decode(dead_name).lower()))
    )
    # depuis le sondage
    .query('timestamp >= "2024-01-15"')
    .query('deadname_count > 0')
    [['user', 'deadname_count']]
)

8